In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/26 21:43:45 WARN Utils: Your hostname, Silvialaptop resolves to a loopback address: 127.0.1.1; using 172.31.73.54 instead (on interface eth0)
24/03/26 21:43:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/26 21:43:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [56]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [57]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [58]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [59]:
import pandas as pd

In [60]:
df_pandas = pd.read_csv('head.csv')

In [61]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [62]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

Integer - 4 bytes Long - 8 bytes

In [63]:
from pyspark.sql import types

In [64]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [65]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [66]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [67]:
df = df.repartition(24)

In [68]:
df.write.parquet('fhvhv/2021/01/')

24/03/26 22:21:08 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/26 22:21:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/26 22:21:13 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/26 22:21:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/26 22:21:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [85]:
df = spark.read.parquet('fhvhv/2021/01/')

In [86]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [87]:
df.select('pickup_datetime','dropoff_datetime','PULocationID','DOLocationID') \
.filter(df.hvfhs_license_num == 'HV0003') \
.show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-01 16:17:16|2021-01-01 16:35:20|         153|         167|
|2021-01-04 15:44:57|2021-01-04 15:56:21|         127|         244|
|2021-01-03 14:05:52|2021-01-03 14:20:29|         183|          20|
|2021-01-01 09:27:50|2021-01-01 09:46:50|         162|         106|
|2021-01-04 10:53:41|2021-01-04 11:53:07|          18|          18|
|2021-01-04 22:29:41|2021-01-04 22:51:07|          76|         215|
|2021-01-02 14:52:09|2021-01-02 15:12:01|         220|          32|
|2021-01-01 07:31:49|2021-01-01 07:46:16|         167|          94|
|2021-01-02 22:35:51|2021-01-02 23:07:17|          71|         130|
|2021-01-01 15:38:12|2021-01-01 15:58:56|         222|          61|
|2021-01-05 02:08:44|2021-01-05 02:26:05|         181|          17|
|2021-01-01 03:56:09|2021-01-01 04:06:33|       

In [88]:
from pyspark.sql import functions as F

In [89]:
df \
.withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
.withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+-------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|pickup_date|dropoff_datet|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+-------------+
|           HV0003|              B02884|2021-01-01 16:17:16|2021-01-01 16:35:20|         153|         167|   NULL| 2021-01-01|   2021-01-01|
|           HV0003|              B02882|2021-01-04 15:44:57|2021-01-04 15:56:21|         127|         244|   NULL| 2021-01-04|   2021-01-04|
|           HV0005|              B02510|2021-01-04 14:29:31|2021-01-04 14:54:56|         138|          49|   NULL| 2021-01-04|   2021-01-04|
|           HV0003|              B02864|2021-01-03 14:05:52|2021-01-03 14:20:29|         183|          20|   NULL| 2021-01-03|   2021-01-03|
|           H

In [90]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02884|2021-01-01 16:17:16|2021-01-01 16:35:20|         153|         167|   NULL|
|           HV0003|              B02882|2021-01-04 15:44:57|2021-01-04 15:56:21|         127|         244|   NULL|
|           HV0005|              B02510|2021-01-04 14:29:31|2021-01-04 14:54:56|         138|          49|   NULL|
|           HV0003|              B02864|2021-01-03 14:05:52|2021-01-03 14:20:29|         183|          20|   NULL|
|           HV0003|              B02867|2021-01-01 09:27:50|2021-01-01 09:46:50|         162|         106|   NULL|
|           HV0005|              B02510|2021-01-05 11:26:21|2021-01-05 11:55:13|

In [91]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [92]:
crazy_stuff('B02884')

's/b44'

In [93]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [94]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  s/b44| 2021-01-01|  2021-01-01|         153|         167|
|  e/b42| 2021-01-04|  2021-01-04|         127|         244|
|  e/9ce| 2021-01-04|  2021-01-04|         138|          49|
|  e/b30| 2021-01-03|  2021-01-03|         183|          20|
|  e/b33| 2021-01-01|  2021-01-01|         162|         106|
|  e/9ce| 2021-01-05|  2021-01-05|         216|         265|
|  e/b48| 2021-01-04|  2021-01-04|          18|          18|
|  e/9ce| 2021-01-05|  2021-01-05|          42|         207|
|  e/b3b| 2021-01-04|  2021-01-04|          76|         215|
|  e/acc| 2021-01-02|  2021-01-02|         220|          32|
|  e/b3b| 2021-01-01|  2021-01-01|         167|          94|
|  e/b32| 2021-01-02|  2021-01-02|          71|         130|
|  e/b38| 2021-01-01|  2021-01-01|         222|          61|
|  e/9ce| 2021-01-03|  2

In [96]:
!head -n 10 head.csv